# Training and Visualizing a Decision Tree

[iris dataset](https://scikit-learn.org/stable/datasets/toy_dataset.html#iris-dataset)

Attributes:
- sepal length in cm (がく片)
- sepal width in cm
- petal length in cm (花びら)
- petal width in cm

class:
- Iris-Setosa
- Iris-Versicolour
- Iris-Virginica

In [1]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier

iris=load_iris()
X=iris.data[:, 2:] # petal length, widthのみ使用
y=iris.target


In [2]:
tree_clf=DecisionTreeClassifier(max_depth=2)
tree_clf.fit(X,y)

DecisionTreeClassifier(max_depth=2)

In [3]:
import os
from sklearn.tree import export_graphviz

image_path='.'
export_graphviz(
    tree_clf,
    out_file=os.path.join(image_path, 'iris_tree.dot'),
    feature_names=iris.feature_names[2:],
    class_names=iris.target_names,
    rounded=True,
    filled=True
)


In [1]:
from graphviz import Source

Source.from_file(os.path.join(image_path, 'iris_tree.dot'))

ModuleNotFoundError: No module named 'graphviz'

# Making Predictions

ジニ不純度

$$
G_i=1-\sum_{k=1}^np_{i,k}^2
$$

# Estimating Class Probabilities

In [14]:
tree_clf.predict_proba([[5,1.5]])

array([[0.        , 0.90740741, 0.09259259]])

In [15]:
tree_clf.predict([[5,1.5]])

array([1])

# The CART Training Algorithm

Classification and Regression Tree

$$
J(k,t_k)=\frac{m_{left}}{m}G_{left}+\frac{m_{right}}{m}G_{right}
$$

$G_{left , right}$:不純度

$m_{left , right}$:インスタンス数

# Regression

cost function for regression

$$
J(k,t_k)=\frac{m_{left}}{m}MSE_{left}+\frac{m_{right}}{m}MSE_{right}
$$


In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg=DecisionTreeRegressor(max_depth=2)
tree_reg.fit(X, y)